In [ ]:
import pandas as pd
import random
import json
import pyodbc

def load_data_from_csv(csv_file):
    # Chargement des données à partir du fichier CSV avec pandas
    data = pd.read_csv(csv_file)
    return data

def split_data(data, json_file, db_connection_str, db_table, json_percentage, db_percentage):
    # Mélanger les lignes de manière aléatoire
    data = data.sample(frac=1).reset_index(drop=True)
    print("Data shuffled")

    # Calculer le nombre de lignes nécessaires pour chaque destination
    total_rows = len(data)
    json_rows = int(total_rows * json_percentage)
    db_rows = int(total_rows * db_percentage)
    csv_rows = total_rows - json_rows - db_rows

    # Création du fichier JSON
    json_data = data.iloc[:json_rows]
    json_data.to_json(json_file, orient='records', lines=True)
    print(f"{json_rows} rows saved to JSON file: {json_file}")

    # Création du fichier CSV
    csv_data = data.iloc[json_rows:json_rows + csv_rows]
    csv_data.to_csv('output.csv', index=False)
    print(f"{csv_rows} rows saved to CSV file: output.csv")

    # Transfert vers la base de données SQL Server
    db_data = data.iloc[json_rows + csv_rows:]
    print("Connecting to the database...")
    db_conn = pyodbc.connect(db_connection_str)
    cursor = db_conn.cursor()
    # Supprimer la table si elle existe déjà
    cursor.execute(f"IF OBJECT_ID('{db_table}', 'U') IS NOT NULL DROP TABLE {db_table}")
    
    print('database connected successfuly.')

    # Créer une nouvelle table pour les données
    create_table_sql = f"CREATE TABLE {db_table} ({','.join([f'{col_name} NVARCHAR(MAX)' for col_name in data.columns])})"
    cursor.execute(create_table_sql)
    print(f"Table {db_table} created in the database")

    # Insérer les données dans la table
    for _, row in db_data.iterrows():
        values = ",".join([f"'{str(value)}'" for value in row.values])
        cursor.execute(f"INSERT INTO {db_table} VALUES ({values})")
    print(f"Table {db_table} created in the database")
    db_conn.commit()
    db_conn.close()
    print(f"{db_rows} rows saved to the database table: {db_table}")

# Provide the necessary information here
csv_file = 'lottery.csv'
json_file = 'output.json'
db_server = 'LAPTOP-JESJKN69\SQLEXPRESS'
db_database = 'lot_win'
db_table = 'winnings'
json_percentage = 0.3
db_percentage = 0.3

# Charger les données à partir du fichier CSV
data = load_data_from_csv(csv_file)
print(f"Loaded {len(data)} rows from the CSV file.")

# Construire la chaîne de connexion SQL Server
db_connection_str = f"DRIVER={{SQL Server}};SERVER={db_server};DATABASE={db_database};Trusted_Connection=yes;"

# Diviser et sauvegarder les données
split_data(data, json_file, db_connection_str, db_table, json_percentage, db_percentage)
